In [1]:
import syft as sy
import torch
from torch import nn, optim

In [2]:
hook = sy.TorchHook(torch)

In [3]:
bob = sy.VirtualWorker(hook, id='bob')
alice = sy.VirtualWorker(hook, id='alice')

In [4]:
raw_data = torch.tensor([[0., 0], [0, 1], [1, 0], [1, 1]], requires_grad=True)
target = torch.tensor([[0.],[1], [1], [1]], requires_grad=True)

In [5]:
data_bob = raw_data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [6]:
data_alice = raw_data[2:].send(alice)
target_alice = target[2:].send(alice)

In [7]:
bob._objects

{7080003931: tensor([[0., 0.],
         [0., 1.]], requires_grad=True), 98615546761: tensor([[0.],
         [1.]], requires_grad=True)}

In [8]:
alice._objects

{15258922227: tensor([[1., 0.],
         [1., 1.]], requires_grad=True), 34162026590: tensor([[1.],
         [1.]], requires_grad=True)}

In [9]:
dataset = [(data_bob, target_bob), (data_alice, target_alice)]

In [14]:
def train(iterations=20):
    
    model = nn.Linear(2, 1)
    optimizer = optim.SGD(params=model.parameters(), lr=0.1)
    
    for i in range(iterations):

        for _data, _target in dataset:

            # Move the model to the various locations
            model = model.send(_data.location)

            # Remove the accumulating Gradients
            optimizer.zero_grad()

            # Make the prediction
            pred = model(_data)

            # Calculate the Loss
            loss = ((pred- _target)**2).sum()

            # Perform the back propagations
            loss.backward()

            # Update the weights
            optimizer.step()

            # Get Smarter model back
            model = model.get()
            print(loss.get())

In [16]:
train()

tensor(0.8437, requires_grad=True)
tensor(0.1043, requires_grad=True)
tensor(0.6050, requires_grad=True)
tensor(0.0012, requires_grad=True)
tensor(0.5179, requires_grad=True)
tensor(0.0044, requires_grad=True)
tensor(0.4484, requires_grad=True)
tensor(0.0120, requires_grad=True)
tensor(0.3929, requires_grad=True)
tensor(0.0216, requires_grad=True)
tensor(0.3482, requires_grad=True)
tensor(0.0319, requires_grad=True)
tensor(0.3121, requires_grad=True)
tensor(0.0423, requires_grad=True)
tensor(0.2828, requires_grad=True)
tensor(0.0524, requires_grad=True)
tensor(0.2589, requires_grad=True)
tensor(0.0619, requires_grad=True)
tensor(0.2393, requires_grad=True)
tensor(0.0707, requires_grad=True)
tensor(0.2231, requires_grad=True)
tensor(0.0787, requires_grad=True)
tensor(0.2098, requires_grad=True)
tensor(0.0859, requires_grad=True)
tensor(0.1987, requires_grad=True)
tensor(0.0923, requires_grad=True)
tensor(0.1895, requires_grad=True)
tensor(0.0981, requires_grad=True)
tensor(0.1818, requi